<a href="https://colab.research.google.com/github/Priyojit02/classification_fashion_images_RESNET_MODEl/blob/main/notebook1d80e7cf7a_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        continue

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pywt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
import os
import pandas as pd

def w2d(img, mode='haar', level=2):
    imArray = img
    # Datatype conversions
    # Convert to grayscale
    imArray = cv2.cvtColor(imArray, cv2.COLOR_RGB2GRAY)
    # Convert to float
    imArray = np.float32(imArray)
    imArray /= 255
    # Compute coefficients
    coeffs = pywt.wavedec2(imArray, mode, level=level)

    # Process Coefficients
    coeffs_H = list(coeffs)
    coeffs_H[0] *= 0

    # Reconstruction
    imArray_H = pywt.waverec2(coeffs_H, mode)
    imArray_H *= 255
    imArray_H = np.uint8(imArray_H)

    return imArray_H

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:

# Function to load data from a given directory
def load_data(directory_path, csv_path):
    df = pd.read_csv(csv_path)

    X = []  # List to store images
    y = []  # List to store labels

    if not os.path.exists(directory_path):
        raise ValueError(f"The specified directory '{directory_path}' does not exist.")

    for i in range(df.shape[0]):
        img_path = os.path.join(directory_path, df.iloc[i]['file_name'])
        label = df.iloc[i]['label']

        # Read the image using cv2.imread
        img = cv2.imread(img_path)
        # Resize the image
        img_resized = cv2.resize(img, (150, 150))
        # Perform wavelet transform
       # img_wvlet = w2d(img_resized, mode='db2', level=5)

        # Stack the single-channel image to create a three-channel image
      #  img_wvlet_rgb = np.stack([img_wvlet, img_wvlet, img_wvlet], axis=-1)

       # img_wvlet_rgb = img_wvlet_rgb / 255

        if i % 100 == 0:
            print(i, img_resized.shape)

        X.append(img_resized)
        y.append(label)

    if not X or not y:
        raise ValueError(f"No images found in the specified directory '{directory_path}'.")
    return np.array(X), np.array(y)

data_directory = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train'
csv_path = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train.csv'
X, y = load_data(data_directory, csv_path)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


0 (150, 150, 3)
100 (150, 150, 3)
200 (150, 150, 3)
300 (150, 150, 3)
400 (150, 150, 3)
500 (150, 150, 3)
600 (150, 150, 3)
700 (150, 150, 3)
800 (150, 150, 3)
900 (150, 150, 3)
1000 (150, 150, 3)
1100 (150, 150, 3)
1200 (150, 150, 3)
1300 (150, 150, 3)
1400 (150, 150, 3)
1500 (150, 150, 3)
1600 (150, 150, 3)
1700 (150, 150, 3)
1800 (150, 150, 3)
1900 (150, 150, 3)
2000 (150, 150, 3)
2100 (150, 150, 3)
2200 (150, 150, 3)
2300 (150, 150, 3)
2400 (150, 150, 3)
2500 (150, 150, 3)
2600 (150, 150, 3)
2700 (150, 150, 3)
2800 (150, 150, 3)
2900 (150, 150, 3)
3000 (150, 150, 3)
3100 (150, 150, 3)
3200 (150, 150, 3)
3300 (150, 150, 3)
3400 (150, 150, 3)
3500 (150, 150, 3)
3600 (150, 150, 3)
3700 (150, 150, 3)
3800 (150, 150, 3)
3900 (150, 150, 3)
4000 (150, 150, 3)
4100 (150, 150, 3)
4200 (150, 150, 3)
4300 (150, 150, 3)
4400 (150, 150, 3)
4500 (150, 150, 3)
4600 (150, 150, 3)
4700 (150, 150, 3)
4800 (150, 150, 3)
4900 (150, 150, 3)
5000 (150, 150, 3)
5100 (150, 150, 3)
5200 (150, 150, 3)
5300 

In [ ]:
y.unique()

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
import pandas as pd

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)

# Create ResNet50-based model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(650, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(300, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(124, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

# Output layer
num_classes = len(set(y_train))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Learning rate scheduling
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)

# Model checkpointing
checkpoint_filepath = 'best_model.h5'
model_checkpoint = callbacks.ModelCheckpoint(
    checkpoint_filepath,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# Train the model with data augmentation
datagen.fit(X_train)  # Fit the ImageDataGenerator on your training data (only for normalization purposes)
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[lr_scheduler, model_checkpoint]
)

# Load the best model
model = tf.keras.models.load_model(checkpoint_filepath)

# Make predictions on the test set
predictions = model.predict(X_test)
predicted_labels = tf.argmax(predictions, axis=1).numpy()

# Evaluate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f'Accuracy: {accuracy}')

Epoch 1/30
489/489 [==============================] - ETA: 0s - loss: 1.9138 - accuracy: 0.3496
Epoch 1: val_loss improved from inf to 1.44485, saving model to best_model.h5
489/489 [==============================] - 136s 203ms/step - loss: 1.9138 - accuracy: 0.3496 - val_loss: 1.4448 - val_accuracy: 0.5154 - lr: 1.0000e-04
Epoch 2/30
489/489 [==============================] - ETA: 0s - loss: 1.3278 - accuracy: 0.5144
Epoch 2: val_loss did not improve from 1.44485
489/489 [==============================] - 97s 197ms/step - loss: 1.3278 - accuracy: 0.5144 - val_loss: 1.5181 - val_accuracy: 0.4897 - lr: 1.0000e-04
Epoch 3/30
489/489 [==============================] - ETA: 0s - loss: 1.1298 - accuracy: 0.5667
Epoch 3: val_loss improved from 1.44485 to 1.03465, saving model to best_model.h5
489/489 [==============================] - 99s 203ms/step - loss: 1.1298 - accuracy: 0.5667 - val_loss: 1.0346 - val_accuracy: 0.5821 - lr: 1.0000e-04
Epoch 4/30
489/489 [==============================]

In [ ]:
abs_train_path = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train.csv'
abs_train_directory = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/train'
abs_test_path = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/sample_submission.csv'
abs_test_directory = '/kaggle/input/extracting-attributes-from-fashion-images-jan-2024/test'

df2 = pd.read_csv(abs_test_path)

X_test = []

# Resize and convert to grayscale for the test set
for i in range(df2.shape[0]):
        img_path = os.path.join(abs_test_directory, df2.iloc[i]['file_name'])
        label = df2.iloc[i]['label']

        # Read the image using cv2.imread
        img = cv2.imread(img_path)
        # Resize the image
        img_resized = cv2.resize(img, (150, 150))
        # Perform wavelet transform

        if i % 100 == 0:
            print(i, img_resized.shape)
        X_test.append(img_resized)


X_test_np = np.array(X_test)


0 (150, 150, 3)
100 (150, 150, 3)
200 (150, 150, 3)
300 (150, 150, 3)
400 (150, 150, 3)
500 (150, 150, 3)
600 (150, 150, 3)
700 (150, 150, 3)
800 (150, 150, 3)
900 (150, 150, 3)
1000 (150, 150, 3)
1100 (150, 150, 3)
1200 (150, 150, 3)
1300 (150, 150, 3)
1400 (150, 150, 3)
1500 (150, 150, 3)
1600 (150, 150, 3)
1700 (150, 150, 3)
1800 (150, 150, 3)
1900 (150, 150, 3)
2000 (150, 150, 3)
2100 (150, 150, 3)
2200 (150, 150, 3)
2300 (150, 150, 3)
2400 (150, 150, 3)
2500 (150, 150, 3)
2600 (150, 150, 3)
2700 (150, 150, 3)
2800 (150, 150, 3)
2900 (150, 150, 3)
3000 (150, 150, 3)
3100 (150, 150, 3)
3200 (150, 150, 3)
3300 (150, 150, 3)
3400 (150, 150, 3)
3500 (150, 150, 3)
3600 (150, 150, 3)
3700 (150, 150, 3)
3800 (150, 150, 3)
3900 (150, 150, 3)
4000 (150, 150, 3)
4100 (150, 150, 3)
4200 (150, 150, 3)
4300 (150, 150, 3)
4400 (150, 150, 3)
4500 (150, 150, 3)
4600 (150, 150, 3)
4700 (150, 150, 3)
4800 (150, 150, 3)
4900 (150, 150, 3)
5000 (150, 150, 3)
5100 (150, 150, 3)
5200 (150, 150, 3)
5300 

In [ ]:
predictions = model.predict(X_test_np)
predicted_labels = tf.argmax(predictions, axis=1).numpy()

180/180 [==============================] - 5s 30ms/step


In [ ]:
import pandas as pd

# Create a DataFrame with the file_name and label columns
df_output = pd.DataFrame({'file_name': df2["file_name"], 'label': predicted_labels})

# Save the DataFrame to a CSV file
df_output.to_csv('output9.csv', index=False)